In [1]:
import csv
import pandas as pd
from rich import print
import pretty_errors
import numpy as np

pretty_errors.replace_stderr()
df = pd.read_excel("sc-eval.xlsx", sheet_name="eval")

placeholders = {"na": r"\scriptsize\color{gray}\raisebox{1.5pt}{n/a}\hspace{2em}", "SKIP": " "}

kernel_cols = list(df.columns)[3:]
kernel_cols = kernel_cols[:3] + kernel_cols[4:6] + kernel_cols[9:]
softwares = list(df['software'].unique())
baselines = ["tf-8280", "aj-v100", "aj-v100"]
softwares_with_baselines = list(zip(softwares, baselines))
#devices = list(df['device'].unique())
data = list(df['datum'].unique())

corresponding_colors = dict(zip(
    ['up-6148', 'b2-7742', 'tf-8280', 'aj-a100', 'aj-v100', 'aj-p100', 'lm-3080', 'aj-8000', 'tf-5000'],
    ["white!80!R", "black", "gray!40", "Bv", "Y", "none", "Bv!60", "gray!20", "gray!10"]))

print(df['device'].unique())
print(kernel_cols)

In [6]:
zip_table_start = r"\begin{tabular}{ @{} r *{11}{ r@{} } }"
zip_table_title = r"& \EVALTITLE{          } & \EVALTITLE{gather}  & \EVALTITLE{         } & \EVALTITLE{Huffman}  & \EVALTITLE{zip:}        & \EVALTITLE{Huffman}  & \EVALTITLE{scatter} & \EVALTITLE{       } & \EVALTITLE{unzip:}       \\[-.6ex]"
zip_table_title+= r"& \EVALTITLE{dual-quant} & \EVALTITLE{outlier} & \EVALTITLE{histogram} & \EVALTITLE{encoding} & \EVALTITLE{all kernels} & \EVALTITLE{decoding} & \EVALTITLE{outlier} & \EVALTITLE{reconst} & \EVALTITLE{all kernels}  \\"
zip_col_start   = 0
zip_col_end     = len(kernel_cols)

unzip_table_start = r"\begin{tabular}{ @{} r *{4}{ r@{} } }"
unzip_table_title = r"& \EVALTITLE{Huffman}  & \EVALTITLE{scatter} & \EVALTITLE{       } & \EVALTITLE{unzip:} \\"
unzip_table_title+= r"& \EVALTITLE{decoding} & \EVALTITLE{outlier} & \EVALTITLE{reconst} & \EVALTITLE{all kernels} \\"
unzip_col_start   = 5
unzip_col_end     = len(kernel_cols)


def gen_content_v1(table_start, table_title, col_start, col_end):
    content = []; content.append(table_start); content.append(table_title)

    for sw,baseline in softwares_with_baselines[:-1]:
        for datum in data:
            on_this_datum = df.query(f'software == "{sw}" & datum == "{datum}"')
            pivot = on_this_datum.query(f'device == "{baseline}"')
            devices = list(on_this_datum['device'].unique())
            
            for dev in devices:
                exp = on_this_datum.query(f'device == "{dev}"')
                color = corresponding_colors[dev]
                short_dev = dev.split('-')[1]
                
                if short_dev in ["p100", "6148"]:
                    continue
                
                items_this_exp = [r"\raisebox{1.5pt}{\scriptsize " + short_dev.upper() + "}"]
                
                for i, k in enumerate(kernel_cols[col_start:col_end]):
                    try:
                        if k == "codebook":
                            item = "\\TIME{{{:.2e}}}{{}}{{}}{{white}}".format(float(exp[k]))
                        else:
                            pivot_perf = float(pivot[k])
                            
                            this_perf = float(exp[k])
                            ratio = this_perf / pivot_perf
                            
                            base_draw_len = np.log(pivot_perf)
                            draw_len = base_draw_len * ratio 
                            draw_len*= 1.5 if i == col_end - col_start - 1 else 1
                            
                            bar = "BARdefault"
                            if short_dev in ["3080", "5000", "8000"]:
                                bar = "BARpattern"
                            if short_dev in ["v100"]:
                                bar = "BARreference"
                            item = f"\{bar}{{{this_perf:.01f}}}{{{ratio:.2f}}}{{{draw_len}}}{{{color}}}" ## #1 val, #2 ratio, #3 draw-length, #4 color
                    except ValueError:
                        item = placeholders[pivot[k].to_string(index=False).strip()]
                    items_this_exp.append(str(item))
                line = " & ".join([str(i) for i in items_this_exp]) + "\t" + "\\\\[-1.0ex]"
                content.append(line)
            content.append(r"\\[-1ex]")
    content.append(r"\end{tabular}")

    return content

In [10]:
content= gen_content_v1(zip_table_start, zip_table_title, zip_col_start, zip_col_end)
open("sample_eval_tab.tex", 'w').write("\n".join(content))

!pdflatex --interaction=batchmode sample_eval && open sample_eval.pdf

This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
